# Item 68: Use Threads for Blocking I/O; Avoid for Parallelism



In [14]:
def factorize(number): 
    for i in range(1, number + 1):
        if number % i == 0:
            yield i
            
import time 

numbers = [7775876, 6694411, 5038540, 5426782,
           9934740, 9168996, 5271226, 8288002,
           9403196, 6678888, 6776096, 9582542,
           7107467, 9633726, 5747908, 7613918]
start = time.perf_counter()

for number in numbers:
    list(factorize(number))

end = time.perf_counter()
delta = end - start
print(f"Serial takes {delta: .5f} seconds!")

from threading import Thread

class FactorizeThread(Thread):
    def __init__(self, number):
        super().__init__()
        self.number = number 
        
    def run(self):
        self.factors = list(factorize(self.number))
        
tstart = time.perf_counter()
threads = []
for n in numbers:
    thread = FactorizeThread(n)
    thread.start()
    threads.append(thread)
    
for t in threads:
    t.join()

tdelta = time.perf_counter() - tstart
print(f"Threading takes {tdelta: .5f} seconds!")



Serial takes  4.42402 seconds!
Threading takes  4.32483 seconds!


In [34]:
import select
import socket
from threading import Thread, Event, Lock
import time

def slow_system_call():
    select.select([socket.socket()], [], [], 3)
    
# start = time.perf_counter()
# print("准备执行!")
# for _ in range(5):
#     slow_system_call()

# print("看到我就说明我没有被卡住!")

# end = time.perf_counter()
# delta = end - start 
# print(f"Took {delta: .3f} seconds")
# start = time.perf_counter()
# threads = []
# for _ in range(5):
#     thread = Thread(target=slow_system_call)
#     thread.start()
#     threads.append(thread)

# for i in range(5):
#     print(f"Do other calcs! - {i}")
    
# for t in threads:
#     t.join()

# delta = time.perf_counter() - start
# print(f"Took {delta: .3f} seconds !!")

# 让我们去掉't.join()'的阻塞
all_tasks_done = Event()
completed_count = 0
counter_lock = Lock()

class FactorizeThread(Thread):
    def __init__(self, counter_lock):
        super().__init__()
        self.counter_lock = counter_lock
        
    def run(self):
        global completed_count
        slow_system_call()
        
        print(f"子线程 {completed_count} 完成了任务")
        with self.counter_lock:
            completed_count += 1
            if completed_count == 5:
                all_tasks_done.set()

threads = []
for i in range(5):
    thread = FactorizeThread(counter_lock)
    thread.start()
    threads.append(thread)
    
while not all_tasks_done.is_set():
    print(f"主线程开始闹着玩 -- {completed_count}")
    time.sleep(0.5)
    
    
print("while 结束了！！！！！！！！")




主线程开始闹着玩 -- 0
主线程开始闹着玩 -- 0
主线程开始闹着玩 -- 0
主线程开始闹着玩 -- 0
主线程开始闹着玩 -- 0
主线程开始闹着玩 -- 0
while 结束了！！！！！！！！
